In [ ]:
## AGENDA
## - Centrality

In [ ]:
#from py2neo import Graph
#graph = Graph("bolt://neo4j.sktai.io:30074", auth=("neo4j","!neo4j00"))

In [ ]:
# https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases/tag/3.5.0.1
#mv ./apoc-3.5.0.1-all.jar $Neo4J_HOME/plugins

In [1]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [2]:
#gph_conn = Neo4jConnection(uri="bolt://neo4j.sktai.io:30074", user="neo4j", pwd="!neo4j00")
#gph_conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="!neo4j00")
gph_conn = Neo4jConnection(uri="bolt://127.0.0.1:7687", user="neo4j", pwd="!neo4j00")

In [3]:
gph_conn.query("""
               CALL db.schema();
               """)

[<Record nodes=[<Node id=-1 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>] relationships=[<Relationship id=-1 nodes=(<Node id=-1 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>, <Node id=-1 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>) type='LINKED_TO' properties={}>]>]

In [4]:
gph_conn.query("""
          MATCH (n)
          DETACH DELETE n
          """)

[]

In [ ]:
## SET DATA

In [5]:
gph_conn.query("""
               CREATE (A:Node {name: "A"})
               CREATE (B:Node {name: "B"})
               CREATE (C:Node {name: "C"})
               CREATE (D:Node {name: "D"})
               CREATE (E:Node {name: "E"})
               
               CREATE (A)-[:LINKED_TO {weight: 10}]->(B)
               CREATE (A)-[:LINKED_TO {weight: 33}]->(C)
               CREATE (A)-[:LINKED_TO {weight: 35}]->(D)
               CREATE (B)-[:LINKED_TO {weight: 20}]->(C)
               CREATE (C)-[:LINKED_TO {weight: 28}]->(D)
               CREATE (C)-[:LINKED_TO {weight: 6 }]->(E)
               CREATE (D)-[:LINKED_TO {weight: 40}]->(E)
               """)

[]

In [ ]:
## DEGREE CENTRALITY via Native

In [13]:
gph_conn.query("""
               MATCH (n:Node)
               OPTIONAL MATCH (n)-[r:LINKED_TO]->()
               RETURN n.name, count(r) as outgoingDegree 
               ORDER BY outgoingDegree DESC
               """)

[<Record n.name='A' outgoingDegree=3>,
 <Record n.name='C' outgoingDegree=2>,
 <Record n.name='B' outgoingDegree=1>,
 <Record n.name='D' outgoingDegree=1>,
 <Record n.name='E' outgoingDegree=0>]

In [15]:
gph_conn.query("""
               MATCH (n:Node)
               OPTIONAL MATCH (n)<-[r:LINKED_TO]-()
               RETURN n.name, count(r) as ingoingDegree 
               ORDER BY ingoingDegree DESC
               """)

[<Record n.name='C' ingoingDegree=2>,
 <Record n.name='D' ingoingDegree=2>,
 <Record n.name='E' ingoingDegree=2>,
 <Record n.name='B' ingoingDegree=1>,
 <Record n.name='A' ingoingDegree=0>]

In [ ]:
## via GDS

In [11]:
gph_conn.query("""
               CALL gds.graph.create("graph_out", "Node", "LINKED_TO")
               """)

[<Record graphName='graph' nodeProjection={'Node': {'properties': {}, 'label': 'Node'}} relationshipProjection={'LINKED_TO': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {}}} nodeCount=5 relationshipCount=7 createMillis=5>]

In [12]:
gph_conn.query("""
               CALL gds.alpha.degree.stream("graph_out")
               YIELD nodeId, score 
               RETURN gds.util.asNode(nodeId).name as nodeName, score 
               ORDER BY score DESC
               """)

[<Record nodeName='A' score=3.0>,
 <Record nodeName='C' score=2.0>,
 <Record nodeName='B' score=1.0>,
 <Record nodeName='D' score=1.0>,
 <Record nodeName='E' score=0.0>]

In [17]:
gph_conn.query("""
               CALL gds.graph.create("graph_in", "Node", {LINKED_TO: { relationship: "LINKED_TO", orientation: "REVERSE" }})
               """)

[<Record graphName='graph_in' nodeProjection={'Node': {'properties': {}, 'label': 'Node'}} relationshipProjection={'LINKED_TO': {'orientation': 'REVERSE', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {}}} nodeCount=5 relationshipCount=7 createMillis=3>]

In [18]:
gph_conn.query("""
               CALL gds.alpha.degree.stream("graph_in")
               YIELD nodeId, score 
               RETURN gds.util.asNode(nodeId).name as nodeName, score 
               ORDER BY score DESC
               """)

[<Record nodeName='C' score=2.0>,
 <Record nodeName='D' score=2.0>,
 <Record nodeName='E' score=2.0>,
 <Record nodeName='B' score=1.0>,
 <Record nodeName='A' score=0.0>]

In [22]:
gph_conn.query("""
               CALL gds.graph.create("graph_und", "Node", {LINKED_TO: { relationship: "LINKED_TO", orientation: "UNDIRECTED" }})
               """)

[<Record graphName='graph_und' nodeProjection={'Node': {'properties': {}, 'label': 'Node'}} relationshipProjection={'LINKED_TO': {'orientation': 'UNDIRECTED', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {}}} nodeCount=5 relationshipCount=14 createMillis=4>]

In [23]:
gph_conn.query("""
               CALL gds.alpha.degree.stream("graph_und")
               YIELD nodeId, score 
               RETURN gds.util.asNode(nodeId).name as nodeName, score 
               ORDER BY score DESC
               """)

[<Record nodeName='C' score=4.0>,
 <Record nodeName='A' score=3.0>,
 <Record nodeName='D' score=3.0>,
 <Record nodeName='B' score=2.0>,
 <Record nodeName='E' score=2.0>]